In [ ]:
import torch
import geospaNN
import numpy as np
import time
import pandas as pd

def f5(X): return (10*np.sin(np.pi*X[:,0]*X[:,1]) + 20*(X[:,2]-0.5)**2 + 10*X[:,3] +5*X[:,4])/6

sigma = 1
phi = 3
tau = 0.01
theta = torch.tensor([sigma, phi / np.sqrt(2), tau])

p = 5; funXY = f5

t = np.empty(0)
size_vec = np.empty(0)
epoch_vec = np.empty(0)


for n in [200, 500, 1000, 2000, 5000, 10000, 20000, 50000, 100000]:
    print(n)
    for rand in range(1, 5 + 1):
        print(rand)
        nn = 20
        batch_size = 50

        torch.manual_seed(2023 + rand)
        X, Y, coord, cov, corerr = geospaNN.Simulation(n, p, nn, funXY, theta, range = [0,1])
        data = geospaNN.make_graph(X, Y, coord, nn)
        train_loader, data_train, data_val, data_test = geospaNN.split_data(X, Y, coord, batch_size = batch_size, neighbor_size = 20,
                                                                         test_proportion = 0.2)

        start_time = time.time()
        mlp = torch.nn.Sequential(
                    torch.nn.Linear(p, 5),
                    torch.nn.ReLU(),
                    torch.nn.Linear(5, 1)
                )
        model = geospaNN.nngls(p=p, neighbor_size=nn, coord_dimensions=2, mlp = mlp, theta = torch.tensor([1.5, 5, 0.1]))
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
        Update_init = 0; Update_step = 1; Update_bound = 0.1; patience_half = 5; patience = 10;
        lr_scheduler = geospaNN.LRScheduler(optimizer, patience = patience_half, factor=0.5)
        early_stopping = geospaNN.EarlyStopping(patience=patience, min_delta = 0.001)

        # Training/evaluation loop
        for epoch in range(100):
            print(epoch)
            # Train for one epoch
            model.train()
            if (epoch >= Update_init) & (epoch % Update_step == 0):
                model.theta.requires_grad = True
            else:
                model.theta.requires_grad = False
            for batch_idx, batch in enumerate(train_loader):
                optimizer.zero_grad()
                decorrelated_preds, decorrelated_targets, est = model(batch)
                loss = torch.nn.functional.mse_loss(decorrelated_preds[:batch_size], decorrelated_targets[:batch_size])
                metric = torch.nn.functional.mse_loss(est[:batch_size], batch.y[:batch_size])
                loss.backward()
                optimizer.step()
            # Compute predictions on held-out test test
            model.eval()
            _, _, val_est = model(data_val)
            val_loss = torch.nn.functional.mse_loss(val_est, data_val.y).item()
            lr_scheduler(val_loss)
            early_stopping(val_loss)
            if early_stopping.early_stop:
                print('End at epoch' + str(epoch))
                break
        end_time = time.time()
        usertime = end_time - start_time
        t = np.append(t, usertime)
        size_vec = np.append(size_vec, n)
        epoch_vec = np.append(epoch_vec, epoch)

        df = pd.DataFrame({'time': t,
                   'epoch': epoch_vec,
                   'size': size_vec
                   })

        df.to_csv("/Users/zhanwentao/Documents/Abhi/NN/NN-GLS/running_time.csv")

200
1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
INFO: Early stopping counter 1 of 10
21
INFO: Early stopping counter 2 of 10
22
INFO: Early stopping counter 3 of 10
23
INFO: Early stopping counter 4 of 10
24
INFO: Early stopping counter 5 of 10
25
Epoch 00026: reducing learning rate of group 0 to 5.0000e-03.
INFO: Early stopping counter 6 of 10
26
INFO: Early stopping counter 7 of 10
27
INFO: Early stopping counter 8 of 10
28
INFO: Early stopping counter 9 of 10
29
INFO: Early stopping counter 10 of 10
INFO: Early stopping
End at epoch29
2
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
INFO: Early stopping counter 1 of 10
22
23
24
25
26
INFO: Early stopping counter 1 of 10
27
INFO: Early stopping counter 2 of 10
28
29
INFO: Early stopping counter 1 of 10
30
INFO: Early stopping counter 2 of 10
31
INFO: Early stopping counter 3 of 10
32
INFO: Early stopping counter 4 of 10
33
INFO: Early stopping counter 5 of 10
34
Epoch 00035: reducing learning rate of group 0 to 5.